In [1]:
import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-17 15:30:05--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Risoluzione di www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 2001:660:3203:100:1:0:80:107, 157.159.10.107
Connessione a www-public.tem-tsp.eu (www-public.tem-tsp.eu)|2001:660:3203:100:1:0:80:107|:80... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 25546284 (24M) [application/zip]
Salvataggio in: ‘dataset_tsmc2014.zip’

dataset_tsmc2014.zi 100%[===================>]  24,36M  1,07MB/s    in 24s     

2024-05-17 15:30:29 (1,01 MB/s) - ‘dataset_tsmc2014.zip’ salvato [25546284/25546284]

Archive:  dataset_tsmc2014.zip
   creating: dataset_tsmc2014/
  inflating: dataset_tsmc2014/dataset_TSMC2014_NYC.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_readme.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_TKY.txt  


In [2]:
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [3]:
df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
print(df)

        User ID                  Venue ID         Venue category ID  \
0           470  49bbd6c0f964a520f4531fe3  4bf58dd8d48988d127951735   
1           979  4a43c0aef964a520c6a61fe3  4bf58dd8d48988d1df941735   
2            69  4c5cc7b485a1e21e00d35711  4bf58dd8d48988d103941735   
3           395  4bc7086715a7ef3bef9878da  4bf58dd8d48988d104941735   
4            87  4cf2c5321d18a143951b5cec  4bf58dd8d48988d1cb941735   
...         ...                       ...                       ...   
227423      688  3fd66200f964a52000e71ee3  4bf58dd8d48988d1e7931735   
227424      560  4bca32ff0687ef3be789dbcc  4bf58dd8d48988d16c941735   
227425      945  50a77716e4b0b5a9492f6f56  4bf58dd8d48988d103941735   
227426      671  4514efe0f964a520e7391fe3  4bf58dd8d48988d11d941735   
227427      942  4a1e0ca0f964a520bf7b1fe3  4bf58dd8d48988d116941735   

        Venue category name   Latitude  Longitude  Timezone  \
0       Arts & Crafts Store  40.719810 -74.002581      -240   
1                    

In [ ]:
from datetime import datetime

def str2date(date_str):
    
    parsed_date = datetime.strptime(date_str, "%a %b %d %H:%M:%S")
    print(parsed_date)

In [1]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km
    
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    # Calculate the differences in latitude and longitude
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1
    
    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c
    
    return distance

In [3]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326
